In [ ]:
import json
import re
from os import listdir
import skimage.io as io
from shapely.geometry import Polygon, shape, Point
from osgeo import gdal, osr, ogr, gdalnumeric
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
from shapely.geometry import Polygon
import pylab
from area import area
from datetime import datetime
%matplotlib inline 

In [ ]:
def latlon2pixel(lat, lon, input_raster='', targetsr='', geom_transform=''):
    # type: (object, object, object, object, object) -> object

    sourcesr = osr.SpatialReference()
    sourcesr.ImportFromEPSG(4326)

    geom = ogr.Geometry(ogr.wkbPoint)
    geom.AddPoint(lon, lat)

    if targetsr == '':
        src_raster = gdal.Open(input_raster)
        targetsr = osr.SpatialReference()
        targetsr.ImportFromWkt(src_raster.GetProjectionRef())
    coord_trans = osr.CoordinateTransformation(sourcesr, targetsr)
    if geom_transform == '':
        src_raster = gdal.Open(input_raster)
        transform = src_raster.GetGeoTransform()
    else:
        transform = geom_transform

    x_origin = transform[0]
    # print(x_origin)
    y_origin = transform[3]
    # print(y_origin)
    pixel_width = transform[1]
    # print(pixel_width)
    pixel_height = transform[5]
    # print(pixel_height)
    geom.Transform(coord_trans)
    # print(geom.GetPoint())
    x_pix = (geom.GetPoint()[0] - x_origin) / pixel_width
    y_pix = (geom.GetPoint()[1] - y_origin) / pixel_height

    return (x_pix, y_pix)

def geoPolygonToPixelPolygonWKT(geom, inputRaster, targetSR, geomTransform, breakMultiPolygonGeo=True,
                                pixPrecision=2):
    # Returns Pixel Coordinate List and GeoCoordinateList
    return_array = []
    polygonPixBufferList = []
    polygonPixBufferWKTList = []
    polygonGeoWKTList = []
    if geom.GetGeometryName() == 'POLYGON':
        polygonPix = ogr.Geometry(ogr.wkbPolygon)
        for ring in geom:
            # GetPoint returns a tuple not a Geometry
            ringPix = ogr.Geometry(ogr.wkbLinearRing)

            for pIdx in range(ring.GetPointCount()):
                lon, lat, z = ring.GetPoint(pIdx)
                xPix, yPix = latlon2pixel(lat, lon, inputRaster, targetSR, geomTransform)

                xPix = round(xPix, pixPrecision)
                yPix = round(yPix, pixPrecision)
                ringPix.AddPoint(xPix, yPix)
                
                return_array.append(xPix)
                return_array.append(yPix)

            polygonPix.AddGeometry(ringPix)
        polygonPixBuffer = polygonPix.Buffer(0.0)
        polygonPixBufferList.append([polygonPixBuffer, geom])

    elif geom.GetGeometryName() == 'MULTIPOLYGON':

        for poly in geom:
            polygonPix = ogr.Geometry(ogr.wkbPolygon)
            for ring in poly:
                # GetPoint returns a tuple not a Geometry
                ringPix = ogr.Geometry(ogr.wkbLinearRing)

                for pIdx in range(ring.GetPointCount()):
                    lon, lat, z = ring.GetPoint(pIdx)
                    xPix, yPix = latlon2pixel(lat, lon, inputRaster, targetSR, geomTransform)

                    xPix = round(xPix, pixPrecision)
                    yPix = round(yPix, pixPrecision)
                    ringPix.AddPoint(xPix, yPix)
                    
                    return_array.append(xPix)
                    return_array.append(yPix)

                polygonPix.AddGeometry(ringPix)
            polygonPixBuffer = polygonPix.Buffer(0.0)
            if breakMultiPolygonGeo:
                polygonPixBufferList.append([polygonPixBuffer, poly])
            else:
                polygonPixBufferList.append([polygonPixBuffer, geom])

    for polygonTest in polygonPixBufferList:
        if polygonTest[0].GetGeometryName() == 'POLYGON':
            polygonPixBufferWKTList.append([polygonTest[0].ExportToWkt(), polygonTest[1].ExportToWkt()])
        elif polygonTest[0].GetGeometryName() == 'MULTIPOLYGON':
            for polygonTest2 in polygonTest[0]:
                polygonPixBufferWKTList.append([polygonTest2.ExportToWkt(), polygonTest[1].ExportToWkt()])

    return return_array, polygonPixBufferWKTList

In [ ]:
#initialize data for output json
my_categories = [{"id": 1,"name": "building", "supercategory": "structure"}]
my_info = {"description": "This is stable 1.0 version of the Vegas SpaceNet dataset in COCO format.", "url": "none", "version": "1.0", "year": 2017, "contributor": "Lee Cohn", "date_created": "2017-04-17 00:00:00.000000"}
my_images = []
my_licenses = [{"id": 1,"name": "Vegas-SpaceNet","url": "none"}]
my_annotations = []

In [ ]:
def process_directory(feature_index):
    #Loop through directory of tif files
    for rasterSrc_file in listdir('vegas_tif'):
        
        #open raster and vector
        srcRas_ds = gdal.Open('vegas_tif/'+rasterSrc_file)
        print("Raster file : ",'vegas_tif/'+rasterSrc_file)
        cols = srcRas_ds.RasterXSize
        rows = srcRas_ds.RasterYSize
        

        #Get image number
        image_number_search = re.search('(?<=img)\w+', rasterSrc_file)
        image_number = image_number_search.group(0)
        image_number = int(image_number)
        
        jpg_file = rasterSrc_file.replace('.tif', '.jpg')
        
        
        #create image json entry
        image_json_entry = {"license": 1, "file_name": jpg_file, "coco_url": "none", "height": rows, "width": cols, "date_captured": str(datetime.now()), "flickr_url": "none", "id": image_number}
        my_images.append(image_json_entry)

        vectorSrc_file = 'buildings_AOI_2_Vegas_img'+str(image_number)+'.geojson'
       
        source_ds = ogr.Open('vegas_geojson/'+vectorSrc_file)
        source_layer = source_ds.GetLayer()
        num_features = source_layer.GetFeatureCount()
        
        
        for feature in source_layer:
            my_geom = feature.GetGeometryRef()
            if(my_geom.GetGeometryName() == 'POINT'):
                feature_index = feature_index + 1
                continue
                
            test = geoPolygonToPixelPolygonWKT(my_geom, 'vegas_tif/'+rasterSrc_file, '', '', breakMultiPolygonGeo=True, pixPrecision=2)
            my_list = test[0]
            poly = np.array(my_list).reshape((int(len(my_list)/2), 2))
            my_poly = Polygon(poly)
            x_min, y_min, x_max, y_max = my_poly.bounds
            width = x_max - x_min
            height = y_max - y_min
            my_bbox = [x_min,y_min,width,height]
            
            #create feature json entry
            feature_json_entry = {"id": feature_index,"image_id": image_number,"category_id": 1,"segmentation": [my_list],"area": my_poly.area,"bbox": my_bbox,"iscrowd": 0}
            my_annotations.append(feature_json_entry)
          
            feature_index = feature_index + 1

feature_index = 1            
process_directory(feature_index)
print("DONE")

In [ ]:
json_data = {"info":my_info, "images":my_images, "annotations":my_annotations, "licenses":my_licenses, "categories":my_categories}

with open("vegas_train_instances_2.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)